In [ ]:
import itertools
from src.utilities.similarity_parallel_spark import pyspark_APDS
import pandas as pd
from typing import Tuple, Type, List, Dict, Any

In [ ]:
datasets = ['nfcorpus'] # Choosen datasets
thresholds: List[float] = [0.5, 0.6, 0.7, 0.8, 0.9] # Choosen thresholds
numslices_factor = [1, 2, 5, 10, 15] # Choosen numslices factors N_PARTITIONS
max_workers = [1, 2, 5, 10, 15] # n_executors
considered_docs = 750

# # Download datasets
# datasets_data = {dataset: download_dataset(dataset) for dataset in datasets}
#
# # Pre-process and sample with the original datasets
# pre_processed_data = {dataset: sample_dict(documents_preprocessing(dataset, docs_dict), considered_docs)
#                       for dataset, docs_dict in datasets_data.items()}

In [ ]:

pyspark_results = []

for ds_name, sampled_dict in pre_processed_data.items():

    print(f'\n------------ALL PAIRS DOCUMENTS SIMILARITY - {ds_name}------------')

    for threshold in thresholds:
        print(f'\n--------Running with threshold: {threshold}--------')

        for s_factor, workers in itertools.product(numslices_factor, range(1, max_workers + 1)):
            # PySpark Execution

            print(f'\nPySpark Parallel Execution with {workers} workers and slice factor of {s_factor}')
            sim_doc_ps, ps_res = pyspark_APDS(ds_name=ds_name, sampled_dict=sampled_dict, threshold=threshold, workers=workers, s_factor=s_factor)
            pyspark_results.append(ps_res)
            create_doc_sim_csv(sim_doc_ps, ds_name, threshold, None, workers)
            print(' Done')

        print('\n')


In [ ]:

print('\nSaving pyspark_results')
pd.DataFrame.from_dict(
    dict(zip(range(len(pyspark_results)), pyspark_results)),
    orient='index',
    columns=[
        'ds_name',
        'elapsed',
        'threshold',
        'uniqie_pairs_sim_docs',
        'workers',
        'slice_factor'
    ],
).to_csv('./results/pyspark_results.csv', index=False)
print( 'Done')